In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**IMPORTING THE DATASET**

In [1]:
fashion=keras.datasets.fashion_mnist

**SPLITTING DATASET INTO TRAINING AND TESTING SET**

In [1]:
(train_images,train_labels),(test_images,test_labels)=fashion.load_data()

**NORMALIZING THE PIXELS**

Since the pixels of picture are in range 0 to 255. So, we divide them by 255 to get the values in the range of 0 to 1.

In [1]:
train_images=train_images/255.0
test_images=test_images/255.0

In [1]:
train_images[0].shape

In [1]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

**HYPERPARAMETER TUNING FOR ADDING THE LAYERS WITH BEST PARAMETERS**

In [1]:
def build_model(hp):  
    model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
    ])
  
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
    return model

In [1]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

**SETTING THE VALUES**

In [1]:
tuner_search=RandomSearch(build_model,
                         objective="val_accuracy",
                         max_trials=5,
                         directory="output",
                         project_name="Fashion")

**RUNNING THE MODEL ON THE ABOVE GIVEN PARAMAETRS**

In [1]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.3)

**TO KNOW THE BEST MODEL PARAMETERS**

In [1]:
model=tuner_search.get_best_models(num_models=1)[0]

In [1]:
model.summary()

**TRAINING THE MODEL ON TRAINING DATASET**

In [1]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)